In [87]:
import gym
import ray

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [88]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [89]:
# Initialize Ray
ray.shutdown()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
from rayflow import Rayflow
r = Rayflow.load("test_input.yml", minio_volume_path="./minio_volume/", mlflow_volume_path="./mlflow_volume/",no_ray=True)

HOST INFO {'192.168.1.65': {'ansible_python_interpreter': '~/anaconda3/envs/py37/bin/python', 'ansible_connection': 'local'}}


/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (127.0.0.1) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


In [91]:
r.start()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (192.168.1.65) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-ke

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-11498estutdnq/tmpxj2q21ot TO /home/shyam/.ansible/tmp/ansible-tmp-1601359152.2016077-238049703330371/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601359152.2016077-238049703330371/ /home/shyam/.ansible/tmp/ansible-tmp-1601359152.2016077-238049703330371/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601359152.2016077-238049703330371/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601359152.2016077-238049703330371/ > /dev/null 2>&1 && sleep 0'
ok: [192.168.1.65] => {
    "changed": false,
    "invocation": {
        "module_args": {
            "api_version": "auto",
    

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (192.168.1.65) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts: {}
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-key
        MLFLOW_URL: http://192.168.1.65:5000
        MLFLOW_S3_ENDPOINT_URL: http://192.168.1.65:9001
        AWS_ACCESS_KEY

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-11703qwf2zp8e/tmp4hymslrf TO /home/shyam/.ansible/tmp/ansible-tmp-1601359156.6351469-106119289628208/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601359156.6351469-106119289628208/ /home/shyam/.ansible/tmp/ansible-tmp-1601359156.6351469-106119289628208/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601359156.6351469-106119289628208/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601359156.6351469-106119289628208/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",

<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601359157.494317-265068636858685/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",
            "Args": [
                "server",
                "--backend-store-uri",
                "/mlflow",
                "--host",
                "192.168.1.65",
                "-p",
                "5000",
                "--default-artifact-root",
                "s3://artifacts/"
            ],
            "Config": {
                "AttachStderr": false,
                "AttachStdin": false,
                "AttachStdout": false,
                "Cmd": [
                    "mlflow",
                    "server",
                    "--backend-store-uri",
                    "/mlflow",
                    "--host",
                    "192.168.1.65",
                    "-p",
                    "5000"

In [65]:
ray.init(ignore_reinit_error=True,num_cpus=12, num_gpus=1)

2020-09-28 22:47:26,608	INFO resource_spec.py:231 -- Starting Ray with 31.69 GiB memory available for workers and up to 15.87 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-28 22:47:27,044	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.65',
 'raylet_ip_address': '192.168.1.65',
 'redis_address': '192.168.1.65:6379',
 'object_store_address': '/tmp/ray/session_2020-09-28_22-47-26_607226_107824/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-28_22-47-26_607226_107824/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-09-28_22-47-26_607226_107824'}

In [80]:
from minerl_rllib.envs import register
register()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [170]:
from ray.rllib.agents.ppo import PPOTrainer

train_name = "mlflow_train"
mlflow_logging_config = {
    "run_name":"rayflow_intial_test_run", 
    "run_tags":{},
    "experiment_name":"MULTIPLE_WORKER_TEST",
    "create_new_experiment": True
}
trainer_config = {
         "env": "MineRLNavigateDenseVectorObf-v0",
         "num_gpus":1,
         "num_workers": 1,
         "num_envs_per_worker": 2,
         "framework": "torch",
         'monitor':True, 
}
saver_config = {
        "checkpoint_on_end": True,
        "checkpoint_freq": 2
}
stop = {
        "training_iteration": 3,
}
tune_args = {
        "checkpoint_at_end": True,
        "checkpoint_freq": 2,
}

config = {
    "trainer_config":trainer_config,
    "logging_config": mlflow_logging_config
}

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [171]:
PPOTrainer.default_resource_request(trainer_config).to_json()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'cpu': 1,
 'gpu': 1,
 'memory': 0,
 'object_store_memory': 0,
 'extra_cpu': 1,
 'extra_gpu': 0,
 'extra_memory': 0,
 'extra_object_store_memory': 0,
 'custom_resources': {},
 'extra_custom_resources': {}}

In [83]:
from rayflow.tune import Tune
t = Tune()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [84]:
import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [85]:
run_id, output_config, experiment_analysis = t.run_training(
               trainer=PPOTrainer, 
               name=train_name, 
               stop=stop, 
               mlflow_logging_config=mlflow_logging_config, 
               trainer_config=trainer_config,
               saver_config=saver_config,
               tune_args=tune_args,
               )

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Trial name,status,loc
MLFlowTrainer_b8c3c_00000,RUNNING,


2020-09-28 22:50:47,648	WARNING worker.py:1134 -- The actor or task with ID ffffffffffffffff876814b80100 is pending and cannot currently be scheduled. It requires {CPU: 5.000000}, {GPU: 1.000000} for execution and {CPU: 5.000000}, {GPU: 1.000000} for placement, but this node only has remaining {node:192.168.1.65: 1.000000}, {GPUType:RTX: 1.000000}, {CPU: 4.000000}, {memory: 31.689453 GiB}, {object_store_memory: 10.937500 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.
(pid=3666) 2020-09-28 22:50:47,889	INFO (unknown file):0 -- gc.collect() freed 8 refs in 0.1435978180015809 seconds2020-09-28 22:50:47,898	INFO (unknown file):0 -- gc.collect() freed 5025 refs in 0.15218408199871192 seconds

(pid=3647) 2020-09-2

2020-09-28 22:53:00,478	ERROR trial_runner.py:523 -- Trial MLFlowTrainer_b8c3c_00000: Error processing event.
Traceback (most recent call last):
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 471, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 430, in fetch_result
    result = ray.get(trial_future[0], DEFAULT_GET_TIMEOUT)
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/worker.py", line 1538, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TypeError): ray::MLFlowTrainer.train() (pid=3652, ip=192.168.1.65)
  File "python/ray/_raylet.pyx", line 479, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 432, in ray._raylet.execute_task.function_executor
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/tune/trainable

Trial name,status,loc
MLFlowTrainer_b8c3c_00000,ERROR,
Trial name,# failures,error file
MLFlowTrainer_b8c3c_00000,1,/home/shyam/ray_results/mlflow_train/MLFlowTrainer_0_2020-09-28_22-50-471r6021ik/error.txt


Trial name,status,loc
MLFlowTrainer_b8c3c_00000,ERROR,
Trial name,# failures,error file
MLFlowTrainer_b8c3c_00000,1,/home/shyam/ray_results/mlflow_train/MLFlowTrainer_0_2020-09-28_22-50-471r6021ik/error.txt


TuneError: ('Trials did not complete', [MLFlowTrainer_b8c3c_00000])

(pid=raylet) E0928 22:53:00.629817  3627  3627 process.cc:434] Failed to kill process 3663 with error system:3: No such process
(pid=3665) Attempted to send kill command to minecraft process and failed.
(pid=3665) *** Aborted at 1601358781 (unix time) try "date -d @1601358781" if you are using GNU date ***
(pid=3665) PC: @                0x0 (unknown)
(pid=3665) *** SIGSEGV (@0x7f3c5affc9d0) received by PID 6559 (TID 0x7f407db62740) from PID 1526712784; stack trace: ***
(pid=3665)     @     0x7f407d7538a0 (unknown)
(pid=3665)     @     0x7f407d749bd8 __GI___pthread_timedjoin_ex
(pid=3665)     @     0x7f407b3dd2d3 std::thread::join()
(pid=3665)     @     0x7f407b90e493 ray::gcs::GlobalStateAccessor::Disconnect()
(pid=3665)     @     0x7f407b7adfbc __pyx_pw_3ray_7_raylet_19GlobalStateAccessor_5disconnect()
(pid=3665)     @     0x55c9fb561ea1 _PyMethodDef_RawFastCallKeywords
(pid=3665)     @     0x55c9fb56214f _PyMethodDescr_FastCallKeywords
(pid=3665)     @     0x55c9fb5bdfa9 _PyEval_Eva

In [76]:
mlflow_logging_config = {
    "run_name":"rayflow_intial_test_run", 
    "run_tags":{},
    "experiment_name":"DUDE",
    "create_new_experiment": False
}

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [77]:
config = {
    "trainer_config":  trainer_config,
    "logging_config": mlflow_logging_config,
    "trainer": PPOTrainer,
}

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [78]:
mlflow_trainer = t.restore_trainer(PPOTrainer, trainer_config, mlflow_logging_config, run_id="9f59b245843740cfabef6d7b0a7406dc", checkpoint_path="checkpoint_3", artifact_dir="./mlflow_artifacts/")

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-09-28 22:48:48,728	WARNING deprecation.py:30 -- DeprecationWarning: `use_pytorch` has been deprecated. Use `framework=torch` instead. This will raise an error in the future!
2020-09-28 22:48:48,729	WARNING deprecation.py:30 -- DeprecationWarning: `eager` has been deprecated. Use `framework=tfe` instead. This will raise an error in the future!
/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.wa

In [72]:
mlflow_trainer

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [1]:
import gym
env = gym.make("MineRLNavigateDenseVectorObf-v0")
episode_reward = 0
done = False
obs = env.reset()
while not done:
    action = ppo_trainer.compute_action(obs)
    obs, reward, done, info = env.step(action)
    episode_reward += reward
env.close()

UnregisteredEnv: No registered env with id: MineRLNavigateDenseVectorObf-v0

In [56]:
r.stop()

INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts: {}
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-key
        MLFLOW_URL: http://192.168.1.65:5000
        MLFLOW_S3_ENDPOINT_URL: http://192.168.1.65:9001
        AWS_ACCESS_KEY

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-2255bsqvy187/tmpg6ak256_ TO /home/shyam/.ansible/tmp/ansible-tmp-1601358376.2089603-78843411538990/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601358376.2089603-78843411538990/ /home/shyam/.ansible/tmp/ansible-tmp-1601358376.2089603-78843411538990/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601358376.2089603-78843411538990/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601358376.2089603-78843411538990/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "changed": true,
    "invocation": {
        "module_args": {
            "api_version": "auto",
      

### Register MineRL Gym Environment to RLlib

In [ ]:
from minerl_rllib.envs import register

In [ ]:
register()

In [ ]:
from ray import tune
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples

import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples
# config = ppo.DEFAULT_CONFIG.copy()
# config["num_gpus"] = 1
# config["num_workers"] = 1
# config["num_envs_per_worker"] = 1
# config["eager"] = False
# trainer = ppo.PPOTrainer(config=config, env="MineRLNavigateDenseVectorObf-v0")


tune.run(PPOTrainer,
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker": 1,
                 "eager": False,
                 "use_pytorch": True,
#                  'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 }
         ,stop={"training_iteration": 3})  

In [ ]:
r.stop()